In [ ]:
# Table 'playlist' loaded into DataFrame. Shape: (11, 3)
# Table 'video' loaded into DataFrame. Shape: (510, 14)
# Table 'sqlite_sequence' loaded into DataFrame. Shape: (1, 2)

import sqlite3
import pandas as pd

# SQLite 데이터베이스 연결
conn = sqlite3.connect('db_pycon_youtube_videos.db')

In [ ]:
# 데이터베이스의 모든 테이블 이름 가져오기
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# 각 테이블을 DataFrame으로 변환
dataframes = {}

for table_name in tables:
    table_name = table_name[0]
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    dataframes[table_name] = df
    print(f"Table '{table_name}' loaded into DataFrame. Shape: {df.shape}")

# 연결 종료
conn.close()

In [ ]:
# 각 DataFrame 확인
for table_name, df in dataframes.items():
    print(f"\nPreview of '{table_name}' DataFrame:")
    display(df.head())
    print(f"Columns: {df.columns.tolist()}")

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound, VideoUnavailable
import time

# SQLite 데이터베이스 연결
conn = sqlite3.connect('db_pycon_youtube_videos.db')
cursor = conn.cursor()

# 새로운 transcript 테이블 생성
cursor.execute('''
    CREATE TABLE IF NOT EXISTS transcript (
        video_id INTEGER PRIMARY KEY,
        transcript TEXT,
        length INTEGER,
        summary TEXT,
        FOREIGN KEY (video_id) REFERENCES video(id)
    )
''')
conn.commit()

# 자막 수집 및 테이블 업데이트 함수
def fetch_and_store_transcript(video_id, video_url):
    try:
        # 유튜브 비디오 ID 추출
        video_id_from_url = video_url.split('v=')[-1]

        # 자막 수집 (한국어 우선, 없으면 기본 언어로 시도)
        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id_from_url, languages=['ko'])
        except NoTranscriptFound:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id_from_url)  # 기본 자막 시도

        transcript_text = " ".join([t['text'] for t in transcript_list])
        
        # 자막 길이
        length = sum([t['duration'] for t in transcript_list])

        # 자막 테이블에 저장
        cursor.execute('''
            INSERT INTO transcript (video_id, transcript, length, summary) 
            VALUES (?, ?, ?, ?)
        ''', (video_id, transcript_text, length, None))
        conn.commit()
        
        print(f"자막 수집 성공: {video_url}")

    except (TranscriptsDisabled, NoTranscriptFound, VideoUnavailable) as e:
        print(f"자막 수집 실패: {video_url} - {str(e)}")
        return False
    except Exception as e:
        print(f"예기치 않은 오류 발생: {video_url}")
        return False
    return True

# video 테이블에서 자막이 없는 비디오 목록 조회
videos_df = pd.read_sql_query('''
    SELECT id, video_url 
    FROM video 
    WHERE id NOT IN (SELECT video_id FROM transcript)
''', conn)

# 자막 수집 및 데이터베이스에 저장
for index, row in videos_df.iterrows():
    success = fetch_and_store_transcript(row['id'], row['video_url'])
    
    # 자막 수집 실패 시 재시도
    if not success:
        time.sleep(5)  # 재시도 전 대기
        fetch_and_store_transcript(row['id'], row['video_url'])

# 연결 종료
conn.close()


In [ ]:
# SQLite 데이터베이스 연결
conn = sqlite3.connect('db_pycon_youtube_videos.db')

# 자막 수집 완료 후 transcript 테이블에서 데이터를 DataFrame으로 읽어오기
transcripts_df = pd.read_sql_query('SELECT * FROM transcript', conn)

# 연결 종료
conn.close()

# 자막 수집 성공한 비디오 개수 출력
success_count = transcripts_df.shape[0]
print(f"자막 수집 성공한 비디오 수: {success_count}")

# 수집된 자막 데이터 출력
transcripts_df